[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gdslab/d2spy/blob/main/docs/guides/notebooks/05_visualizing_projects_with_leafmap.ipynb)
[![Jupyter Notebook](https://img.shields.io/badge/Open%20in%20JuypterHub%20-%20%233776AB?logo=jupyter&logoColor=%23F37626&labelColor=%23F5F5F5)](https://lab.d2s.org/hub/user-redirect/lab/tree/tutorials/05_visualizing_projects_with_leafmap.ipynb)

# Visualizing project locations with leafmap
*This guide will walk you through the steps for visualizing the location of your D2S projects with [leafmap](https://github.com/opengeos/leafmap).*

To get started, you will need to have access to a D2S instance where you have created projects and the open-source library [leafmap](https://github.com/opengeos/leafmap) added to your Python environment.

In [ ]:
# Uncomment and run the following line if working out of Google Colab
# !pip install d2spy
# !pip install leafmap

In [ ]:
import leafmap
import pandas as pd

from d2spy.workspace import Workspace

You must connect to your D2S workspace before you can request any data. The [`Workspace`](https://py.d2s.org/workspace/) module's [`connect`](https://py.d2s.org/workspace/#d2spy.workspace.Workspace.connect) method can be used to login to a D2S instance and connect to your workspace in one go.

*Note: This tutorial uses a D2S instance hosted at https://ps2.d2s.org. You will need to have an account and access to data on this instance to use it. Change the URL if you are self-hosting an instance or using an instance hosted elsewhere.*

In [ ]:
# Connect to D2S workspace
workspace = Workspace.connect("https://ps2.d2s.org", "yourD2Semail@example.com")

The Workspace [`get_projects`](https://py.d2s.org/workspace/#d2spy.workspace.Workspace.get_projects) method will retrieve a list of the projects your account can currently access on the D2S instance.

In [ ]:
# Get list of all your projects
projects = workspace.get_projects()
print(projects[0])

The `projects` variable now contains a list of `Project` objects. You can access attributes such as `title`, `description`, `start_date`, `end_date`, and `field` from a `Project` object. The `field` attribute contains a GeoJSON representation of the field boundary provided during the project's creation.

Next, leafmap will be used to plot markers on a map at the location of each of your projects. It requires the location information to be stored in either a CSV file or pandas DataFrame. The next two cells will create a pandas DataFrame containing your project titles and center coordinates.

In [ ]:
# Now make a dictionary that can be used to create a DataFrame
data = []
# Store the project title and the project center coordinates for each project in the workspace
for project in projects:
    data.append([project.title, project.centroid['x'], project.centroid['y']])

In [ ]:
# Now create a Pandas dataFrame
df = pd.DataFrame(data, columns=['title', 'x', 'y'])

With the project titles and center coordinates now stored in a DataFrame, you can plot them using leafmap. 

Leafmap will cluster nearby project markers. The total number of projects within a cluster will be shown at the center of the cluster symbol. When you hover over a cluster, a bounding box for the cluster's region will appear. Clicking on a cluster will zoom the map to the extent of the cluster's bounding box.

In [ ]:
# Now create a map
Map = leafmap.Map()
Map.add_xy_data(df, x="x", y="y", layer_name="My D2S Projects")
Map

In [ ]:
# Create map displaying project boundary for first project
Map = leafmap.Map()
Map.add_geojson(projects[1].get_project_boundary(), layer_name=projects[1].title, fill_colors=['#daaa00'], zoom_to_layer=True)
Map

We can also retrieve any map layers associated with our project using the project's [`get_map_layers`](https://py.d2s.org/project/#d2spy.models.project.Project.get_map_layers) method. This method will return a list of GeoJSON FeatureCollections objects for each map layer.

In [ ]:
# Create map for displaying map layers
Map = leafmap.Map()
# Use filter_by_title to find project with map layers
demo_projects = projects.filter_by_title("Demo")
if len(demo_projects) > 0:
    project_with_map_layers = demo_projects[0]
    # Returns list of dictionaries in GeoJSON FeatureCollection structure for each project map layer
    map_layers = project_with_map_layers.get_map_layers()
    # Display map layers on map
    for map_layer in map_layers:
        Map.add_geojson(map_layer, fill_colors=["#daaa00"])
Map

Once finished viewing your data, you can revoke your authorization session by logging out.

In [ ]:
# Removes access token from future requests
workspace.logout()